In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import json
import sqlite3


df_orders = pd.read_csv('orders.csv')


df_users = pd.read_json('users.json')


with open('restaurants.sql', 'r') as f:
    sql_script = f.read()


conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.executescript(sql_script)


df_restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

print("Data Loaded Successfully!")

Data Loaded Successfully!


In [3]:

merged_df = pd.merge(df_orders, df_users, on='user_id', how='left')


final_df = pd.merge(merged_df, df_restaurants, on='restaurant_id', how='left')


final_df.to_csv('final_food_delivery_dataset.csv', index=False)

print("Final Dataset Created! Shape:", final_df.shape)
final_df.head()

Final Dataset Created! Shape: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [5]:
print(final_df.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')


In [7]:

gold_members_df = final_df[final_df['membership'] == 'Gold']


city_revenue = gold_members_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)


print("Total Revenue from Gold Members by City:")
print(city_revenue)

Total Revenue from Gold Members by City:
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [8]:

cuisine_aov = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)


print("Average Order Value by Cuisine:")
print(cuisine_aov)


Average Order Value by Cuisine:
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [9]:

user_total_spending = final_df.groupby('user_id')['total_amount'].sum()


distinct_users_count = (user_total_spending > 1000).sum()

print(f"Number of distinct users with total orders > 1000: {distinct_users_count}")

Number of distinct users with total orders > 1000: 2544


In [10]:

bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)


rating_revenue = final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)

print("Total Revenue by Restaurant Rating Range:")
print(rating_revenue)

Total Revenue by Restaurant Rating Range:
rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64


/tmp/ipython-input-745639701.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


In [12]:

gold_members_df = final_df[final_df['membership'] == 'Gold']


city_gold_aov = gold_members_df.groupby('city')['total_amount'].mean().sort_values(ascending=False)

print("Average Order Value (AOV) for Gold Members by City:")
print(city_gold_aov)

Average Order Value (AOV) for Gold Members by City:
city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [13]:

cuisine_summary = final_df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).sort_values(by='distinct_restaurants')


print("Cuisine Summary (Sorted by Number of Restaurants):")
print(cuisine_summary)

Cuisine Summary (Sorted by Number of Restaurants):
         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09


In [15]:

gold_orders_count = len(final_df[final_df['membership'] == 'Gold'])
total_orders_count = len(final_df)


percentage_gold = (gold_orders_count / total_orders_count) * 100

print(f"Total Orders: {total_orders_count}")
print(f"Gold Member Orders: {gold_orders_count}")
print(f"Percentage of Gold Member Orders: {round(percentage_gold)}%")

Total Orders: 10000
Gold Member Orders: 4987
Percentage of Gold Member Orders: 50%


In [18]:

restaurant_stats = final_df.groupby('restaurant_name_y').agg(
    order_count=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)

low_volume_restaurants = restaurant_stats[restaurant_stats['order_count'] < 20]


top_avg_low_volume = low_volume_restaurants.sort_values(by='avg_order_value', ascending=False)

print("Top AOV Restaurants with < 20 orders:")
print(top_avg_low_volume.head())

Top AOV Restaurants with < 20 orders:
                   order_count  avg_order_value
restaurant_name_y                              
Restaurant_294              13      1040.222308
Restaurant_262              18      1029.473333
Restaurant_77               12      1029.180833
Restaurant_193              15      1026.306667
Restaurant_7                16      1002.140625


In [20]:

combination_revenue = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)

print("Total Revenue by Membership and Cuisine Combination:")
print(combination_revenue)

Total Revenue by Membership and Cuisine Combination:
membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64


In [21]:

final_df['order_date'] = pd.to_datetime(final_df['order_date'])


quarterly_revenue = final_df.groupby(final_df['order_date'].dt.quarter)['total_amount'].sum().sort_values(ascending=False)

print("Total Revenue by Quarter:")
print(quarterly_revenue)

Total Revenue by Quarter:
order_date
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64


/tmp/ipython-input-1545494150.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


In [22]:

hyderabad_total_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()


print(f"Total Revenue for Hyderabad: {round(hyderabad_total_revenue)}")

Total Revenue for Hyderabad: 1889367


In [23]:

distinct_users_count = final_df['user_id'].nunique()

print(f"Number of distinct users who placed at least one order: {distinct_users_count}")

Number of distinct users who placed at least one order: 2883


In [24]:

gold_members_df = final_df[final_df['membership'] == 'Gold']


gold_aov = gold_members_df['total_amount'].mean()

print(f"Average Order Value for Gold Members: {gold_aov:.2f}")

Average Order Value for Gold Members: 797.15


In [25]:

high_rated_orders = final_df[final_df['rating'] >= 4.5].shape[0]

print(f"Total orders for restaurants with rating ≥ 4.5: {high_rated_orders}")

Total orders for restaurants with rating ≥ 4.5: 3374


In [26]:

gold_df = final_df[final_df['membership'] == 'Gold']


gold_city_revenue = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
top_gold_city = gold_city_revenue.index[0]
top_gold_revenue = gold_city_revenue.values[0]


order_count = gold_df[gold_df['city'] == top_gold_city].shape[0]

print(f"Top Revenue City for Gold Members: {top_gold_city}")
print(f"Total Revenue in {top_gold_city} (Gold): {top_gold_revenue:.2f}")
print(f"Number of Gold orders in {top_gold_city}: {order_count}")

Top Revenue City for Gold Members: Chennai
Total Revenue in Chennai (Gold): 1080909.79
Number of Gold orders in Chennai: 1337
